In [1]:
import pandas as pd
import numpy as np

In [2]:
dt = pd.read_csv('orgin_proc_1.csv', index_col=0)
dt

,Find,Conc,Y,sum
0,Clinical information : 두부외상 후 후유증 평가\r\n\r\nAx...,1. Encephalomalacic change in both frontal lob...,0,Clinical information : 두부외상 후 후유증 평가\r\n\r\nAx...
1,Clinical information : lung cancer\nAxial T1WI...,1. No change of focal enhancing lesion in left...,0,Clinical information : lung cancer\nAxial T1WI...
2,Clinical information : Multiple Sclerosis\r\n\...,No significant interval change of abnormal hyp...,0,Clinical information : Multiple Sclerosis\r\n\...
3,Clinical information : patient with DLBCL.\r\n...,1. Decreased extent of enhancing mass in the l...,0,Clinical information : patient with DLBCL.\r\n...
4,Clinical information : Transient cerebral isch...,"1. Acute infarctions at right BG, right F-P-T ...",1,Clinical information : Transient cerebral isch...
...,...,...,...,...
6185,Clinical information : s/p Removal of vestibul...,No evidence of remnant mass or remarkable post...,0,Clinical information : s/p Removal of vestibul...
6186,"CI, headache of sudden onset (known UIA).\r\nA...",1. No evidence of acute infarctions.\r\n2. Enc...,0,"CI, headache of sudden onset (known UIA).\r\nA..."
6187,Clinical information : patient with DLBCL.\n\n...,1. Increased size of homogeneous enhancing mas...,0,Clinical information : patient with DLBCL.\n\n...
6188,Clinical information : Lung cancer patient 임.\...,No evidence of intracranial metastasis.\n,0,Clinical information : Lung cancer patient 임.\...


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(dt['sum'], dt['Y'], test_size=0.10, random_state=123)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer(stop_words=['*', '.', ':','(',')',',','+'])
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

X_test_cnt_vect = cnt_vect.transform(X_test)

In [5]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, Y_train)
pred_lr = lr_clf.predict(X_test_cnt_vect)
print(accuracy_score(Y_test, pred_lr), roc_auc_score(Y_test, pred_lr))

0.9854604200323102 0.9616374092009685


C:\anaconda3\envs\bhoon\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
from sklearn.model_selection import GridSearchCV

def logistic_tuning(train_sprs,y,param):
    model = LogisticRegression(random_state=123)
    grid = GridSearchCV(model, param, scoring='roc_auc', cv=5)
    grid.fit(train_sprs, y)
    
    print(grid.best_params_)
    print(grid.best_score_)
    
    return grid.best_estimator_

In [7]:
%%time

param1= {'C':[0.5,0.525, 0.55,0.575, 0.6],
        'max_iter': [1000, 2000, 3000,4000, 5000]} # penalty 는 L2가 고정이라 제거, c가 1인 경우 잘 됌, max_iter가 큰것이 좋음
logistic_tuning(X_train_cnt_vect, Y_train, param = param1)

{'C': 0.525, 'max_iter': 1000}
0.9861306068470072
CPU times: total: 26.7 s
Wall time: 8.94 s


LogisticRegression(C=0.525, max_iter=1000, random_state=123)

In [8]:
%%time
param2= {'C':[0.51,0.525, 0.54],
        'max_iter': [750, 850, 950, 1000, 1050, 1150,1250]}
logistic_tuning(X_train_cnt_vect, Y_train, param = param2)

{'C': 0.54, 'max_iter': 750}
0.986134295179582
CPU times: total: 22.8 s
Wall time: 7.64 s


LogisticRegression(C=0.54, max_iter=750, random_state=123)

In [9]:
%%time
param3= {'C':[0.535, 0.54, 0.545],
        'max_iter': [500,550, 600,650, 700, 750]}
logistic_tuning(X_train_cnt_vect, Y_train, param = param3)

{'C': 0.54, 'max_iter': 500}
0.986134295179582
CPU times: total: 20.1 s
Wall time: 6.74 s


LogisticRegression(C=0.54, max_iter=500, random_state=123)

In [26]:
%%time
param3= {'C':[0.535, 0.54, 0.545],
        'max_iter': [125,130, 140,150,200]}
logistic_tuning(X_train_cnt_vect, Y_train, param = param3)

{'C': 0.54, 'max_iter': 125}
0.986134295179582
CPU times: total: 16.6 s
Wall time: 5.58 s


LogisticRegression(C=0.54, max_iter=125, random_state=123)

In [11]:
model = logistic_tuning(X_train_cnt_vect, Y_train, param = param3)
pred = model.predict_proba(X_test_cnt_vect)

{'C': 0.54, 'max_iter': 200}
0.986134295179582


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf.fit(X_train_tfidf_vect, Y_train)
pred_lr = lr_clf.predict(X_test_tfidf_vect)
print(accuracy_score(Y_test, pred_lr), roc_auc_score(Y_test, pred_lr))

0.9773828756058158 0.8965193704600485


UsageError: Line magic function `%%time` not found.


In [13]:
%%time
param3= {'C':[0.535, 0.54, 0.545],
        'max_iter': [500,550, 600,650, 700, 750]}
logistic_tuning(X_train_tfidf_vect, Y_train, param = param3)

{'C': 0.545, 'max_iter': 500}
0.9844258661766612
CPU times: total: 6.62 s
Wall time: 2.24 s


LogisticRegression(C=0.545, max_iter=500, random_state=123)

In [27]:
%%time
param3= {'C':[0.535, 0.54, 0.545],
        'max_iter': [125,130,140,200]}
logistic_tuning(X_train_tfidf_vect, Y_train, param = param3)

{'C': 0.545, 'max_iter': 125}
0.9844258661766612
CPU times: total: 4.72 s
Wall time: 1.57 s


LogisticRegression(C=0.545, max_iter=125, random_state=123)

가장 높은 점수: 0.9861342 
               -> counterVectorize + logisticRegression + hyperparameter_tuning